In [13]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# OpenAI API 키 설정
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [15]:
# 나이 계산 함수 정의
def calculate_age(birth_date):
    from datetime import datetime
    today = datetime.today()
    birth_date = datetime.strptime(birth_date, '%Y-%m-%d')
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age

In [16]:
# 환율 변환 함수 정의
def convert_currency(amount, from_currency, to_currency, rate) :
    if from_currency == to_currency:
        return amount
    return amount * rate

In [17]:
# BMI 계산 함수 정의
def calculate_bmi(weight, height):
    if height <= 0:
        raise ValueError("Height must be greater than zero.")
    bmi = weight / (height ** 2)
    return round(bmi, 2)

In [ ]:
# 함수 호출 정의
# 툴 스펙
tools = [{
    'type': 'function',
    'name' : 'calculate_age',
    'description': 'Calculate age from birth date in YYYY-MM-DD format.',
    'parameters': {
        'type': 'object',
        'properties': {
            'birth_date': {
                'type': 'string',
                'format': 'date',
                'description': 'Birth date in YYYY-MM-DD format.'
            }
        },
        'required': ['birth_date']
    }
}, {
    'type': 'function',
    'name': 'convert_currency',
    'description': 'Convert currency from one type to another using a given exchange rate.',
    'parameters': {
        'type': 'object',
        'properties': {
            'amount': {
                'type': 'number',
                'description': 'Amount of money to convert.'
            },
            'from_currency': {
                'type': 'string',
                'description': 'Currency code to convert from.'
            },
            'to_currency': {
                'type': 'string',
                'description': 'Currency code to convert to.'
            },
            'rate': {
                'type': 'number',
                'description': 'Exchange rate from from_currency to to_currency.'
            }
        },
        'required': ['amount', 'from_currency', 'to_currency', 'rate']
    }
}, {
    'type': 'function',
    'name': 'calculate_bmi',
    'description': 'Calculate Body Mass Index (BMI) from weight in kg and height in meters.',
    'parameters': {
        'type': 'object',
        'properties': {
            'weight': {
                'type': 'number',
                'description': 'Weight in kilograms.'
            },
            'height': {
                'type': 'number',
                'description': 'Height in meters.'
            }
        },
        'required': ['weight', 'height']
    }
}]

In [23]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# 나이 계산 프롬프트
input_messages = [{
    'role' : 'user',
    'content' : '1992-05-01생의 만 나이는?'
}]

# 모델의 함수 호출 요청
response = client.responses.create(
    model = 'gpt-4.1',
    input = input_messages,
    tools = tools
)

In [ ]:
# 함수 코드 실행
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = calculate_age(args['birth_date'])

# 모델에게 실행 결과 전달
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id' : tool_call.call_id,
    'output' : str(result)
})

# 최종 응답 생성
final_response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(final_response.output_text)

1992년 5월 1일생의 만 나이는 33세입니다.


In [ ]:
# 환율 변환 프롬프트
input_messages = [{
    'role' : 'user',
    'content' : '100달러를 원화로 바꿔줘. 환율은 1330원을 적용해줘.'
}]

# 모델의 함수 호출 요청
response = client.responses.create(
    model = 'gpt-4.1',
    input = input_messages,
    tools = tools
)

In [ ]:
# 함수 코드 실행
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = convert_currency(args['amount'], args['from_currency'], args['to_currency'], args['rate'])

# 모델에게 실행 결과 전달
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id' : tool_call.call_id,
    'output' : str(result)
})

# 최종 응답 생성
final_response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(final_response.output_text)

100달러를 원화로 환전하면, 환율 1330원을 적용했을 때 133,000원이 됩니다.


In [ ]:
# BMI 계산 프롬프트
input_messages = [{
    'role' : 'user',
    'content' : '키 170cm, 몸무게 65kg의 BMI는?'
}]

# 모델의 함수 호출 요청
response = client.responses.create(
    model = 'gpt-4.1',
    input = input_messages,
    tools = tools
)

In [ ]:
# 함수 코드 실행
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = calculate_bmi(args['height'], args['weight'])

# 모델에게 실행 결과 전달
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id' : tool_call.call_id,
    'output' : str(result)
})

# 최종 응답 생성
final_response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(final_response.output_text)

키 170cm(1.7m), 몸무게 65kg인 경우의 BMI를 계산해드리면 다음과 같습니다:

BMI = 22.5

이는 표준체중 범위(18.5~24.9)에 해당합니다. 건강한 체중입니다!
